#### Import Libraries

In [378]:
import pandas as pd
import numpy as np
import os
import re
import string, unicodedata
import contractions
import codecs
import nltk
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
import spacy
import en_core_web_sm
import time
import gensim
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import sys
from gensim.models import Word2Vec
nlp = en_core_web_sm.load()
nlp.max_length = 600000000
import json

#### Define Corpus

In [379]:
def get_esa_text():
    """Extract ESA Text from text files"""
    esa_text = []
    files = os.listdir("G:/Post Construction/ESA_text")
    for file in files:
        with codecs.open("G:/Post Construction/ESA_text/" + file,'r', encoding='utf-8-sig') as corpus:
            input_str = corpus.read()
            esa_text.append(input_str)
    return esa_text

def get_pcmr_text():
    """Extract PCMR Text from text files"""
    pcmr_text = []
    files = os.listdir("G:/Post Construction/PDF_text")
    for file in files:
        with codecs.open("G:/Post Construction/PDF_text/" + file,'r', encoding='utf-8-sig') as corpus:
            input_str = corpus.read()
            pcmr_text.append(input_str)
    return pcmr_text

def combine_text():
    """combine text string from ESA and PCMR text"""
    esa_corpus = get_esa_text()
    pcmr_corpus = get_pcmr_text()
    corpus = esa_corpus + pcmr_corpus
    return corpus

#### Noise Removal and Text Corpus Normalization

In [380]:
def replace_contractions(text):
    """Replace contractions in string of text"""
    return contractions.fix(text)

def remove_non_ascii(text):
    """Remove non-ASCII characters from text string i.e. converting accented characters/letters"""
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    return text

def to_lowercase(text):
    """Convert all characters to lowercase from text string"""
    text = text.lower()
    return text

def lemmatize_text(text):
    """convert word in the text string to its root form"""
    text = nlp(text)
    text = ' '.join([word.lemma_ if word.lemma_ != '-PRON-' else word.text for word in text])
    return text

def remove_special_characters(text, remove_digits = False):
    """Removing non-alphanumeric characters and symbols or even ocasionally numeric characters"""
    pattern = r'[^a-zA-z0-9\s]' if not remove_digits else r'[^a-zA-z\s]'
    text = re.sub(pattern, '', text)
    return text

def remove_stopwords(text):
    """Remove stop words from text string"""
    stopword_list = nltk.corpus.stopwords.words('english')     
    return ' '.join(word for word in text.split() if word not in stopword_list)

def prohibitedWords(text):
    """The list of words to be removed from the SQL database issues table to avoid capturing false positives"""
    text = text.split()
    prohibitedWordList = ['issue', 'become', 'therefore', 'monitor', 'compare', 'observe', 'risk', 'construct', 'part', 'conduct', 'focus', 'prior', 'manage', 'consider', 'moderate', 'condition', 'potential', 'action', 'reassess', 'row', 'impact', 'control', 'management', 'good', 'unique', 'introduce', 'list', 'potentially', 'low', 'establish', 'legislation']
    resultwords  = [word for word in text if word not in prohibitedWordList]
    text = ' '.join(resultwords)
    return text

In [261]:
text = 'compaction ( ) contouring_1 subsidence_1 admixing_1 contouring_1 crowning_1 1 . loss of agricultural capability 2016 â€“ equivalent land capability have be achieve .6 . < s > wetlands</s > scalping'
pattern = '|'.join(['<s>', '</s>',])
text = text.replace(pattern, '')
text = remove_non_ascii(text)
text = to_lowercase(text)
pattern = '|'.join(['<s>', '</s>',])
text = text.replace(pattern, '')
text = remove_special_characters(text, remove_digits= True)
#text = lemmatize_text(text)
text = remove_stopwords(text)
print(text)

compaction contouring_ subsidence_ admixing_ contouring_ crowning_ loss agricultural capability equivalent land capability achieve wetlandss scalping


In [258]:
if re.search(r'\b' + 'subsidence' + r'\b', text):
    print('present')

In [250]:
dict_final

{'soil': ['alleviate',
  'pellet inject',
  'productivity lower',
  'topography class',
  'topsoil',
  'bulk density',
  'testing',
  'plant vigor',
  'notation',
  'sampling',
  'nearly level',
  'soil',
  'discing',
  'admix',
  'description apprev',
  'crop growth',
  'phase',
  'severe',
  'stunted crop',
  'alleviate compaction',
  'productivity',
  'resistance',
  'cultivate varying',
  'admixing',
  'texture',
  'depth',
  'differentiate subsoil',
  'warrant alleviate',
  'obo',
  'break level',
  'shovel resistance',
  'unit',
  'rutting',
  'investigation site',
  'gentle slope',
  'minor',
  'penetration',
  'unit depth',
  'altalis township',
  'reduce',
  'subsoil',
  'physical resistance',
  'organic carbon',
  'saobo',
  'para till',
  'rut',
  'shags topographic',
  'reduced crop'],
 'vegetation': ['compare',
  'management',
  'species',
  'wd woody',
  'control',
  'rare plant',
  'per square',
  'forbs shrub',
  'patch',
  'exhibit excellent',
  'liverwort',
  'plant',

#### Bringing it All Together - Building a Text Normalizer

In [381]:
def normalize_text_corpus(corpus):
    """Normalize each document in the corpus"""
    start_time = time.time()
    normalized_corpus = []
    corpus_size = len(get_pcmr_text())
    for doc in corpus:
        doc = replace_contractions(doc)
        doc = remove_non_ascii(doc)
        doc = to_lowercase(doc)
        doc = lemmatize_text(doc)
        doc = remove_special_characters(doc, remove_digits = True)
        doc = remove_stopwords(doc)
        normalized_corpus.append(doc)
    dur = round(time.time() - start_time)
    print(f"Normalized text from {corpus_size} documents in {dur} seconds ({round(dur / 60, 2)} min or {round(dur / 3600, 2)} hours)")
    return normalized_corpus

#### Further Processing and Tokenization

In [382]:
def sent_to_words(sentences):
    """using Gensim's simple text preprocessing to convert document into a list of tokens, ignoring tokens that are too short or too long"""
    start_time = time.time()
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=False))
    dur = round(time.time() - start_time)
    print(f"Tokenization and further preprocessing completed in {dur} seconds ({round(dur / 60, 2)} min or {round(dur / 3600, 2)} hours)")

In [383]:
normalized_tokens = list(sent_to_words(normalize_text_corpus(get_pcmr_text())))

Normalized text from 584 documents in 2363 seconds (39.38 min or 0.66 hours)
Tokenization and further preprocessing completed in 10 seconds (0.17 min or 0.0 hours)


#### Build Bi-grams

In [384]:
def make_bigrams(normalized_tokens):
    """ create bigrams froms normalized tokens corpus"""
    bigram = gensim.models.Phrases(normalized_tokens, min_count = 18, threshold = 16)
    bigram_mod = gensim.models.phrases.Phraser(bigram)
    return [bigram_mod[doc] for doc in normalized_tokens]

#min_count: ignore all words and bigrams with total collected count lower than this
#threshold represents a score threshold for forming the phrases (higher means fewer phrases). A phrase of words a followed by b is accepted if the score of the phrase is greater than threshold

normalized_tokens_bigrams = make_bigrams(normalized_tokens)
tokens = 0
for i in normalized_tokens_bigrams:
    tokens += len(i)
print(f"Total tokens in the final corpus: {tokens}")

Total tokens in the final corpus: 3438911


#### Define Hyperparameters

In [385]:
# Set values for various parameters
###############################################
feature_size = 100    # word vector 
window_context = 10   # context window size i.e. maximum distance between current and predicted word within a sentence
min_word_count = 36   # Words that appear only once or twice in a billion-word corpus are probably uninteresting typos and garbage. In addition, there’s not enough data to make any meaningful training on those words, so it’s best to ignore them
sample = 1e-3         # The threshold for configuring which higher-frequency words are randomly downsampled, useful range is (0, 1e-5).
learning_rate = 0.01  # the initial learning rate
iterations = 20        # Number of iterations over the corpus

#### Generate Training Data and Model Training

In [386]:
start_time = time.time()
w2v_model = Word2Vec(min_count = min_word_count,
                     window = window_context,
                     size = feature_size,
                     sg = 1,
                     sample = sample,
                     negative = 3,
                     iter = iterations,
                     workers = 1)
w2v_model.build_vocab(normalized_tokens_bigrams)
w2v_model.train(normalized_tokens_bigrams, total_examples=w2v_model.corpus_count, epochs = w2v_model.iter)
dur = round(time.time() - start_time)
print(f"Word2vec model creation and training completed in {dur} seconds ({round(dur / 60, 2)} min or {round(dur / 3600, 2)} hours)")

Word2vec model creation and training completed in 501 seconds (8.35 min or 0.14 hours)


#### Define Target Words

In [387]:
vec_lst = ['physical_environment', 'soil', 'vegetation', 'water', 'fish', 'wetland', 'wildlife', 'species', 'air', 'air_quality', 'acoustic_environment', 'heritage', 'heritage_resource', 'access']
sub_cat_vec_lst = ['erosion', 'coarse_fragment', 'subsidence', 'compaction', 'watercourse', 'invasive', 'plant', 'weed', 'rare', 'stream', 'riparian']
vec_sub_cat = []
vec_lst.extend(sub_cat_vec_lst)
vec_sub_cat.extend(vec_lst)

In [388]:
root_word_dict = {}
for root_word in vec_sub_cat:
    try:
        context_words = w2v_model.wv.most_similar(positive = [root_word],topn = 25)
        root_word_dict[root_word] = context_words
    except:
        root_word_dict[root_word] = 'The word is not in vocabulary'

In [236]:
"""word2vec_df = pd.DataFrame.from_dict(root_word_dict)
word2vec_df.to_csv('word2vec_pcmr_bigrams_sep21.csv', encoding = 'utf-8-sig')"""

"word2vec_df = pd.DataFrame.from_dict(root_word_dict)\nword2vec_df.to_csv('word2vec_pcmr_bigrams_sep21.csv', encoding = 'utf-8-sig')"

In [389]:
def merge_keys(dict, key1, key2):
    """Merge values of similar context words together"""
    for context_word in dict[key1]:
        dict[key2].append(context_word)
    del dict[key1]
    return dict

def append_value(dict, key, value):
    """Append values in the dictionary"""
    dict[key].append(value)
    return dict

def append_key_as_value(dict):
    """Append dictionary key as value"""
    for key in dict:
        dict[key].append((key, 1.0))
    return dict

def remove_underscores_duplicates(dict):
    """Remove underscores from the dictionary keys and values bigrams followed by removing duplicates from values"""
    dict_final = {}
    for key,value in dict.items():
        new_key = key.replace('_', ' ')
        new_value = [value[0].replace('_', ' ') for value in dict[key]]
        dict_final[new_key] = new_value
    return {key:list(set(value)) for key, value in dict_final.items()}

def replace_keys(dict, old_keys, new_keys):
    """Replace some of the keys for the purpose of naming consistency in SQL database"""
    for idx, new_key in enumerate(new_keys):
        dict[new_key] = dict.pop((old_keys)[idx])
    return dict

def remove_dictionary_values(dictionary, vec, context_words):
    """Remove the context words which were incorrectly tagged to VECs in word2vec model"""
    for key, value in dictionary.items():
        if key == vec:
            for word in context_words:
                if word in value:
                    value.remove(word)
    return dict

In [390]:
vegetation_context_words = ['wetland', 'subsidence', 'erosion']
water_context_words = ['erosion']
wildlife_context_words = ['air quality', 'acoustic environment']
air_context_words = ['acoustic environment', 'habitat', 'special status', 'wildlife']
heritage_context_words = ['acoustic environment', 'air quality']
physical_context_words = ['weed', 'admix']
wetlands_context_words = ['vegetation']
acoustic_context_words = ['air quality', 'course fragment', 'wildlife']

In [391]:
if __name__ == '__main__':
    merge_keys(root_word_dict, 'erosion', 'physical_environment')
    merge_keys(root_word_dict, 'coarse_fragment', 'physical_environment')
    merge_keys(root_word_dict, 'subsidence', 'physical_environment')
    merge_keys(root_word_dict, 'compaction', 'soil')
    merge_keys(root_word_dict, 'invasive', 'vegetation')
    merge_keys(root_word_dict, 'plant', 'vegetation')
    merge_keys(root_word_dict, 'weed', 'vegetation')
    merge_keys(root_word_dict, 'rare', 'vegetation')
    merge_keys(root_word_dict, 'watercourse', 'wetland')
    merge_keys(root_word_dict, 'stream', 'wetland')
    merge_keys(root_word_dict, 'riparian', 'wetland')
    merge_keys(root_word_dict, 'heritage_resource', 'heritage')
    merge_keys(root_word_dict, 'air_quality', 'air')
    append_value(root_word_dict, 'physical_environment', ('coarse fragment', 1.0))
    append_value(root_word_dict, 'vegetation', ('plant', 1.0))
    append_value(root_word_dict, 'vegetation', ('invasive', 1.0))
    append_value(root_word_dict, 'wetland', ('watercourse', 1.0))
    append_value(root_word_dict, 'access', ('navigation', 1.0))
    append_key_as_value(root_word_dict)
    dict_final = remove_underscores_duplicates(root_word_dict)
    old_keys = ['physical environment','wetland', 'acoustic environment', 'access']
    new_keys = ['physical', 'wetlands', 'acoustic', 'navigation']
    replace_keys(dict_final, old_keys, new_keys)
    remove_dictionary_values(dict_final, 'vegetation', vegetation_context_words)
    remove_dictionary_values(dict_final, 'water', water_context_words)
    remove_dictionary_values(dict_final, 'wildlife', wildlife_context_words)
    remove_dictionary_values(dict_final, 'air', air_context_words)
    remove_dictionary_values(dict_final, 'heritage', heritage_context_words)
    remove_dictionary_values(dict_final, 'physical', physical_context_words)
    remove_dictionary_values(dict_final, 'wetlands', wetlands_context_words)
    remove_dictionary_values(dict_final, 'acoustic', acoustic_context_words)

In [392]:
from sqlalchemy import create_engine
# Importing environmental variables library that reads from the .env file
from dotenv import load_dotenv

# Loading key-value pairs from the .env file into the OS environment
load_dotenv()

# Reading the key-value pairs from the OS environment
user = os.getenv("DB_USER")
password = os.getenv("DB_PASS")
db_hostname = os.getenv("DB_HOST")
db_name = os.getenv("DB_DATABASE")

# Using those variables in the connection string using "F" strings
conn_string = f"mysql+mysqldb://{user}:{password}@{db_hostname}/{db_name}?charset=utf8mb4"
engine = create_engine(conn_string)

In [393]:
query = "SELECT * FROM issues;"

with engine.connect() as conn:
    df = pd.read_sql(query, conn)
    df = df.applymap(lambda x: x.strip() if type(x)==str else x) # delete whitespaces
    df.vec_pri = df.vec_pri.replace('\s+', ' ', regex=True) # delete extra space between text strings
    df.vec_psec = df.vec_sec.replace('\s+', ' ', regex=True)
    df.vec_pri = df.vec_pri.str.lower()
    df.vec_sec = df.vec_sec.str.lower()
    df['vec_pri'].fillna('', inplace = True)
    df['vec_sec'].fillna('', inplace = True)
    df['vec_pri'] = df['vec_pri'].apply(lemmatize_text)
    df['vec_sec'] = df['vec_sec'].apply(lemmatize_text)
    df['vec_pri'] = df['vec_pri'].apply(prohibitedWords)
    df['vec_sec'] = df['vec_sec'].apply(prohibitedWords)
    
#df.loc[df.vec_pri.str.contains("(?i)physical environment", na = False), 'physical'] = 1

In [394]:
vec_keyword_count = []
vec_keywords = []

for index, row in enumerate(df.itertuples()):
    issue_keyword_count = []
    
    for key, value in dict_final.items():
        counter = 0
        keyword = []
        for vec in value:
            if re.search(r'\b' + vec + r'\b', row.vec_pri):
                keyword.append(vec)
                counter += 1
        issue_keyword_count.append(counter)
        vec_keywords.append(keyword)
        
    if sum(issue_keyword_count) == 0:
        issue_keyword_count = []
        keyword = []
        for key, value in dict_final.items():
            idx = 0
            for vec in value:
                if re.search(r'\b' + vec + r'\b', row.vec_sec):
                    keyword.append(vec)
                    idx += 1
            issue_keyword_count.append(idx)
        vec_keywords.append(keyword)
            
    vec_keyword_count.append(issue_keyword_count)

In [395]:
# Create the pandas DataFrame  
df1 = pd.DataFrame(vec_keyword_count, columns = dict_final.keys()) 
df1 

,soil,vegetation,water,fish,wildlife,species,air,heritage,physical,wetlands,acoustic,navigation
0,0,0,1,0,0,0,0,0,2,0,0,0
1,0,1,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,1,0,0,0
3,0,3,0,0,0,1,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
5185,0,0,0,0,0,0,0,0,0,0,0,0
5186,0,0,0,0,0,0,0,0,0,0,0,0
5187,0,1,0,0,0,0,0,0,1,1,0,0
5188,0,0,0,0,0,0,0,0,0,0,0,0


In [375]:
df2 = pd.concat([df, df1], axis = 1)
df2['VECassigned'] = df2[['soil', 'vegetation', 'water', 'fish', 'wildlife', 'species', 'air','heritage', 'physical', 'wetlands', 'acoustic', 'navigation']].idxmax(axis=1)
for idx, row in df2.iterrows():
    if  df2.loc[idx,'vec_simple'] == "generic":
        df2.loc[idx,'VECassigned'] = "generic"

In [226]:
'''df = pd.read_csv('file.csv', encoding='cp1252')
df.head()

,vec,filing_manual_text
0,soil,Describe general soil characteristics and the ...
1,vegetation,For lands where vegetation may be affected by ...
2,water,Provide a project-specific water use assessmen...
3,fish,Identify fish species and their life stages in...
4,wildlife,"Identify wildlife species of ecological, econo..."


In [227]:
'''df = df.applymap(lambda x: x.strip() if type(x)==str else x) # delete whitespaces
df.filing_manual_text = df.filing_manual_text.replace('\s+', ' ', regex=True) # delete extra space between text strings
df.filing_manual_text = df.filing_manual_text.str.lower()
df['filing_manual_text'] = df['filing_manual_text'].apply(lemmatize_text)
df['filing_manual_text'] = df['filing_manual_text'].apply(prohibitedWords)'''

In [228]:
vec_keyword_count = []
vec_keywords = []

for index, row in enumerate(df.itertuples()):
    issue_keyword_count = []
    
    for key, value in dict_final.items():
        counter = 0
        keyword = []
        for vec in value:
            if re.search(r'\b' + vec + r'\b', row.filing_manual_text):
                keyword.append(vec)
                counter += 1
        issue_keyword_count.append(counter)
        vec_keywords.append(keyword)
            
    vec_keyword_count.append(issue_keyword_count)

vec_keyword_count

[[4, 1, 2, 0, 0, 0, 1, 2, 6, 2, 1, 2],
 [1, 12, 0, 1, 2, 4, 3, 3, 1, 4, 2, 3],
 [3, 1, 6, 1, 0, 0, 1, 1, 1, 5, 1, 0],
 [0, 3, 2, 4, 1, 2, 2, 0, 0, 8, 2, 1],
 [1, 4, 2, 3, 3, 3, 3, 3, 2, 5, 2, 3],
 [1, 3, 0, 3, 3, 4, 2, 4, 2, 3, 3, 1],
 [0, 2, 1, 0, 0, 1, 2, 2, 1, 1, 0, 1],
 [0, 1, 0, 0, 0, 0, 1, 5, 1, 0, 1, 1],
 [2, 1, 3, 2, 0, 0, 1, 1, 5, 7, 0, 0],
 [0, 4, 2, 2, 1, 0, 1, 1, 1, 3, 1, 1],
 [0, 2, 0, 0, 1, 1, 3, 1, 0, 0, 2, 1],
 [0, 1, 0, 0, 0, 0, 1, 0, 0, 2, 0, 2]]

#### visualizing the Words

In [ ]:
similar_words = {search_term: [item[0] for item in w2v_model.wv.most_similar([search_term], topn = 2)] for search_term in ['physical', 'soil', 'erosion', 'vegetation', 'water', 'fish', 'wetland', 'wildlife', 'specie', 'air']}

In [ ]:
def tsne_plot(model):
    "Create TSNE model and plot it"
    words = sum([[k] + v for k, v in similar_words.items()], [])
    wvs = w2v_model.wv[words]
    tsne_model = TSNE(perplexity = 2, n_components = 2, n_iter = 10000, random_state = 0)
    np.set_printoptions(suppress = True)
    T = tsne_model.fit_transform(wvs)
    labels = words
    plt.figure(figsize=(14, 8))
    plt.scatter(T[:, 0], T[:, 1], c = 'orange', edgecolors = 'r')
    for label, x, y in zip(labels, T[:, 0], T[:, 1]):
        plt.annotate(label, xy=(x+1, y+1), xytext=(0, 0), textcoords='offset points')
    return plt.show()

In [ ]:
tsne_plot(w2v_model)